## 

### General notice
- as far as I know, you need tensorflow<=2.12 for the model from teachable machine to load
- arduino code is at the end of this file
- you need to change your serial channel according to where your board is connected to

In [1]:
!pip install pyserial
!pip install opencv-python

  Obtaining dependency information for pyserial from https://files.pythonhosted.org/packages/07/bc/587a445451b253b285629263eb51c2d8e9bcea4fc97826266d186f96f558/pyserial-3.5-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 910.5 kB/s eta 0:00:000:00:01


In [3]:
import tensorflow
tensorflow.version.VERSION
#you need to downgrade your tensorflow if the version is higher than 2.12.0
#2.13 may work, not tested yet

'2.12.0'

In [5]:
from keras.models import load_model  # TensorFlow is required for Keras to work
import cv2  # Install opencv-python
import numpy as np
import serial
import time

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("sorter_real/keras_Model.h5", compile=False)
print("Model Loaded")

# Load the labels
class_names = open("sorter_real/labels.txt", "r").readlines()

arduino = serial.Serial('/dev/tty.usbmodem11301', 9600) #change this based on your own channel
time.sleep(2)  # Wait for the serial connection to establish

# sent a command to the board
arduino.write(b't')  # Sends the character '1' to the Arduino
print("Sent: Test connection")

response = arduino.readline().decode('utf-8').strip()  # Read and decode the response
print(f"Arduino response: {response}")

# CAMERA can be 0 or 1 based on default camera of your computer
camera = cv2.VideoCapture(0)

while True:
    time.sleep(2)
    
    # Grab the webcamera's image.
    ret, image = camera.read()

    # Resize the raw image into (224-height,224-width) pixels
    image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

    # Show the image in a window
    cv2.imshow("Webcam Image", image)

    # Make the image a numpy array and reshape it to the models input shape.
    image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)

    # Normalize the image array
    image = (image / 127.5) - 1

    # Predicts the model
    prediction = model.predict(image,verbose = 0)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]

    status_line = f"\rClass: {class_name[2:].strip()} | Confidence: {str(np.round(confidence_score * 100))[:-2]}% | "
    
    if index == 0:
        arduino.write(b'0')  
        response = arduino.readline().decode('utf-8').strip()
        status_line += f"Status: Empty | Arduino: {response}"
    elif index == 1:  
        arduino.write(b'1')  
        response = arduino.readline().decode('utf-8').strip()
        status_line += f"Status: Ear Plug detected | Arduino: {response}"
    elif index == 2 :
        arduino.write(b'2')
        response = arduino.readline().decode('utf-8').strip()
        status_line += f"Status: Paper ball detected | Arduino: {response}"
    
    # Pad the line with spaces to ensure complete overwrite
    status_line = status_line.ljust(100)
    print(status_line, end='', flush=True)
    
    # Listen to the keyboard for presses.
    keyboard_input = cv2.waitKey(1)
    

    # 27 is the ASCII for the esc key on your keyboard.
    if keyboard_input == 27:
        break

camera.release()
cv2.destroyAllWindows()


Model Loaded
Sent: Test connection


KeyboardInterrupt: 

### arduino code
```C++

void setup() {
  Serial.begin(9600);  // Start serial communication at 9600 baud rate
  pinMode(13, OUTPUT); // Set pin 13 as an output for the built-in LED
}

void loop() {
  if (Serial.available()) {
    char command = Serial.read();  // Read the incoming byte
    if (command == '0') {
      digitalWrite(13, HIGH);  // Turn on the LED
      Serial.println("Connection established");  // Send a response back to Python
    } else if(command == '1'){
      // you can add more operations here e.g. control a servo
        digitalWrite(13, HIGH);
        Serial.println("Class 1 recived");
    } else if(command == '2'){
        digitalWrite(13, HIGH); 
        Serial.println("Class 2 recived");
    }
  }
}


```